In [13]:
import math
import csv
from collections import defaultdict

class Label:
    def __init__(self, w_x, w_y, path, visited):
        self.w_x = w_x          # Sum of positive weights (x dimension)
        self.w_y = w_y          # Sum of absolute negative weights (y dimension)
        self.path = path        # Path as list of nodes
        self.visited = visited  # Frozen set of visited nodes

    def __repr__(self):
        return f"Label(w_x={self.w_x:.2f}, w_y={self.w_y:.2f}, path={self.path})"

def load_graph_from_csv(filename):
    """Loads graph from CSV with format: u,v,weight"""
    graph = defaultdict(list)
    max_weight = 0
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for row in reader:
            u, v, weight = row
            weight = float(weight)
            
            # Convert to 2D weight representation
            if weight >= 0:
                wx, wy = weight, 0.0
                max_weight = max(max_weight, weight)
            else:
                wx, wy = 0.0, abs(weight)
            
            graph[u].append((v, wx, wy))
    return graph, max_weight

def additive_fptas_shortest_path(csv_file, source, target, epsilon):
    # Load graph and get maximum positive weight
    graph, W = load_graph_from_csv(csv_file)
    nodes = list(graph.keys())
    n = len(nodes)
    if n == 0:
        return float('inf'), []
    
    # Calculate discretization parameter
    delta = epsilon / (n-1) if n > 1 else epsilon
    max_interval = int((n-1)*W/delta) + 1 if W > 0 else 1

    # Initialize data structures
    labels = defaultdict(dict)  # {node: {interval: Label}}
    initial_visited = frozenset([source])
    labels[source][0] = Label(0.0, 0.0, [source], initial_visited)

    # Main label propagation loop
    for _ in range(n-1):
        new_labels = defaultdict(dict)
        
        for u in labels:
            for interval in labels[u]:
                current_label = labels[u][interval]
                
                for (v, wx_e, wy_e) in graph.get(u, []):
                    # Skip already visited nodes
                    if v in current_label.visited:
                        continue
                    
                    # Calculate new weights
                    new_wx = current_label.w_x + wx_e
                    new_wy = current_label.w_y + wy_e
                    
                    # Calculate interval using uniform discretization
                    interval_new = int(new_wx // delta)
                    
                    # Update visited nodes
                    new_visited = current_label.visited.union(frozenset([v]))
                    
                    # Create new path
                    new_path = current_label.path + [v]
                    
                    # Create new label
                    new_label = Label(new_wx, new_wy, new_path, new_visited)
                    
                    # Update if better than existing label in this interval
                    existing = new_labels[v].get(interval_new)
                    if not existing or new_label.w_y > existing.w_y:
                        new_labels[v][interval_new] = new_label

        # Merge new labels into main structure
        for node in new_labels:
            for interval in new_labels[node]:
                if interval not in labels[node] or \
                   new_labels[node][interval].w_y > labels[node][interval].w_y:
                    labels[node][interval] = new_labels[node][interval]

    # Find best path to target
    min_value = float('inf')
    best_path = []
    for interval in labels.get(target, {}):
        label = labels[target][interval]
        current_value = label.w_x - label.w_y
        if current_value < min_value:
            min_value = current_value
            best_path = label.path

    return min_value, best_path

# Example usage
if __name__ == "__main__":
    CSV_FILE = "graph_data.csv"
    SOURCE = "n0"
    TARGET = "n29"  # Update with your target node
    EPSILON = 0.2   # Example epsilon value
    
    min_value, path = additive_fptas_shortest_path(CSV_FILE, SOURCE, TARGET, EPSILON)
    
    if path:
        print(f"Optimal value: {min_value:.2f}")
        print(f"Path: {' -> '.join(path)}")
    else:
        print("No valid path exists")

Optimal value: -27.00
Path: n0 -> n15 -> n24 -> n6 -> n28 -> n25 -> n20 -> n19 -> n23 -> n18 -> n8 -> n16 -> n17 -> n2 -> n29
